In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision

In [2]:
cwd = os.getcwd()
print(cwd)

c:\IDEA\OverFeat


In [3]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader

from torchsummaryX import summary
from tensorboardX import SummaryWriter

In [4]:
print(torch.__version__)

device = ("cuda" if torch.cuda.is_available() else "cpu") # device 정의
print(device)

1.10.2+cu113
cuda


In [5]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3080'

In [6]:
class OverFeat_accurate(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()

        # train with 221x221 5 random crops and their horizontal filps
        # mini- batches of size 128
        # initialized weight randomly with mu=0, sigma=1x10^-2
        # SGD, momentum=0.6, l2 weight decay of 1x10^-5
        # learning rate 5x10^-2, decay by 0.5 after (30, 50, 60, 70, 80) epochs
        # Dropout on FCN?? -> dropout before classifier conv layer

        self.feature_extractor = nn.Sequential(
            # no contrast normalization is used
            # max polling with non-overlapping
            # 1st and 2nd layer stride 2 instead of 4

            # 1st
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=7, stride=2),  # (b x 96 x 108 x 108)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3),  # (b x 96 x 36 x 36)

            # 2nd
            nn.Conv2d(96, 256, 7, stride= 1),  # (b x 256 x 30 x 30)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (b x 256 x 15 x 15)

            # 3rd
            nn.Conv2d(256, 512, 3, padding=1),  # (b x 512 x 15 x 15)
            nn.ReLU(),

            # 4th
            nn.Conv2d(512, 512, 3, padding=1),  # (b x 512 x 15 x 15)
            nn.ReLU(),

            # 5th
            nn.Conv2d(512, 1024, 3, padding=1),  # (b x 1024 x 15 x 15)
            nn.ReLU(),

            # 6th
            nn.Conv2d(1024, 1024, 3, padding=1),  # (b x 1024 x 15 x 15)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3),  # (b x 1024 x 5 x 5)
        )

        # fully connecyed layers implemented as a convolution layers
        self.classifier = nn.Sequential(
            # 7th
            nn.Dropout(p=0.5, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=4096, kernel_size=5),
            nn.ReLU(),

            # 8th
            nn.Dropout(p=0.5, inplace=True),
            nn.Conv2d(4096, 4096, 1),
            nn.ReLU(),

            # 9th
            nn.Conv2d(4096, num_classes, 1)
        )

        self.init_weight()  # initialize weight

    def init_weight(self):
        for layer in self.feature_extractor:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)

    def forward(self, x):
        """
        Pass the input through the net.
        Args:
            x (Tensor): input tensor
        Returns:
            output (Tensor): output tensor
        """
        x = self.feature_extractor(x)
        return self.classifier(x).squeeze()

In [7]:
class OverFeat_fast(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()

        # train with 221x221 5 random crops and their horizontal filps
        # mini- batches of size 128
        # initialized weight randomly with mu=0, sigma=1x10^-2
        # SGD, momentum=0.6, l2 weight decay of 1x10^-5
        # learning rate 5x10^-2, decay by 0.5 after (30, 50, 60, 70, 80) epochs
        # Dropout on FCN?? -> dropout before classifier conv layer

        self.feature_extractor = nn.Sequential(
            # no contrast normalization is used
            # max polling with non-overlapping
            # 1st and 2nd layer stride 2 instead of 4

            # 1st
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),  # (b x 96 x 56 x 56)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (b x 96 x 28 x 28)

            # 2nd
            nn.Conv2d(96, 256, 5, stride= 1),  # (b x 256 x 24 x 24)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (b x 256 x 12 x 12)

            # 3rd
            nn.Conv2d(256, 512, 3, padding=1),  # (b x 512 x 12 x 12)
            nn.ReLU(),

            # 4th
            nn.Conv2d(512, 1024, 3, padding=1),  # (b x 1024 x 12 x 12)
            nn.ReLU(),

            # 5th
            nn.Conv2d(1024, 1024, 3, padding=1),  # (b x 1024 x 12 x 12)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (b x 1024 x 6 x 6)
        )

        # fully connecyed layers implemented as a convolution layers
        self.classifier = nn.Sequential(
            # 6th
            nn.Dropout(p=0.5, inplace=False),
            nn.Conv2d(in_channels=1024, out_channels=3072, kernel_size=6),
            nn.ReLU(),

            # 7th
            nn.Dropout(p=0.5, inplace=False),
            nn.Conv2d(3072, 4096, 1),
            nn.ReLU(),

            # 8th
            nn.Conv2d(4096, num_classes, 1)
        )

        self.init_weight()  # initialize weight

    def init_weight(self):
        for layer in self.feature_extractor:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)

    def forward(self, x):
        """
        Pass the input through the net.
        Args:
            x (Tensor): input tensor
        Returns:
            output (Tensor): output tensor
        """
        x = self.feature_extractor(x)
        return self.classifier(x).squeeze()

In [8]:
path_img_train = "../ILSVRC2012/ILSVRC2012_img_train/"
path_img_val = "../ILSVRC2012/ILSVRC2012_img_val/"

path_log = "tblog/"
path_checkpoint = "checkpoints/"
if not(os.path.exists(path_log)):
    os.makedirs(path_log)
if not(os.path.exists(path_checkpoint)):
    os.makedirs(path_checkpoint)

In [9]:
seed = torch.initial_seed()
print(f'Used seed : {seed}')

Used seed : 25335229804900


In [10]:
tbwriter = SummaryWriter(log_dir=path_log)

In [11]:
Fast = True

num_epochs = 100
batch_size = 128

In [12]:
if(Fast):
    overfeat = OverFeat_fast(num_classes=1000).to(device)
    summary(overfeat, torch.zeros((128,3,231,231),device=device))
else:
    overfeat = OverFeat_accurate(num_classes=1000).to(device)
    summary(overfeat, torch.zeros((128,3,221,221),device=device))

# overfeat = torch.nn.parallel.DataParallel(overfeat, device_ids=)

                                         Kernel Shape         Output Shape  \
Layer                                                                        
0_feature_extractor.Conv2d_0          [3, 96, 11, 11]    [128, 96, 56, 56]   
1_feature_extractor.ReLU_1                          -    [128, 96, 56, 56]   
2_feature_extractor.MaxPool2d_2                     -    [128, 96, 28, 28]   
3_feature_extractor.Conv2d_3          [96, 256, 5, 5]   [128, 256, 24, 24]   
4_feature_extractor.ReLU_4                          -   [128, 256, 24, 24]   
5_feature_extractor.MaxPool2d_5                     -   [128, 256, 12, 12]   
6_feature_extractor.Conv2d_6         [256, 512, 3, 3]   [128, 512, 12, 12]   
7_feature_extractor.ReLU_7                          -   [128, 512, 12, 12]   
8_feature_extractor.Conv2d_8        [512, 1024, 3, 3]  [128, 1024, 12, 12]   
9_feature_extractor.ReLU_9                          -  [128, 1024, 12, 12]   
10_feature_extractor.Conv2d_10     [1024, 1024, 3, 3]  [128, 102

C:\Anaconda3\envs\IDEA_torch\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


In [13]:
if(Fast):
    cropsize = 231
else:
    cropsize = 221

In [14]:
dataset_train = datasets.ImageFolder(path_img_train, transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(cropsize),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
]))
dataset_val = datasets.ImageFolder(path_img_val, transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(cropsize),
    transforms.ToTensor()
]))

In [15]:
dataloader_train = DataLoader(
    dataset=dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=6,
    pin_memory=True,
    drop_last=True
)
dataloader_val = DataLoader(
    dataset=dataset_val,
    batch_size=batch_size,
    shuffle=True,
    num_workers=6,
    pin_memory=True,
    drop_last=True
)

In [16]:
optimizer = optim.SGD(
    params=overfeat.parameters(),
    momentum=0.6,
    weight_decay=1e-5,
    lr=5e-2
)

lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[30, 50, 60, 70, 80], gamma=0.5)

In [17]:
overfeat.train()
step = 1

for epoch in range(num_epochs):
    for imgs, classes in dataloader_train:
        imgs, classes = imgs.to(device), classes.to(device)

        output = overfeat(imgs)
        loss = F.cross_entropy(output, classes)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if(step % 1e3 == 0):
            with torch.no_grad():
                _, preds = torch.max(output, 1)
                accuracy = torch.sum(preds == classes)

                print(f'Epoch: {epoch + 1} \tStep: {step} \tLoss: {loss.item():.4f} \tAcc: {accuracy.item()}')
                tbwriter.add_scalar('loss', loss.item(), step)
                tbwriter.add_scalar('accuracy', accuracy.item(), step)

                for name, parameter in overfeat.named_parameters():
                    if parameter.grad is not None:
                        avg_grad = torch.mean(parameter.grad)
                        # print(f'\t{name} - grad_avg: {avg_grad}')
                        tbwriter.add_scalar(f'grad_avg/{name}', avg_grad.item(), step)
                        tbwriter.add_histogram(f'grad/{name}', parameter.grad.cpu().numpy(), step)
                    if parameter.data is not None:
                        avg_weight = torch.mean(parameter.data)
                        # print(f'\t{name} - param_avg: {avg_weight}')
                        tbwriter.add_histogram(f'weight/{name}', parameter.data.cpu().numpy(), step)
                        tbwriter.add_scalar(f'weight_avg/{name}', avg_weight.item(), step)

                overfeat.eval()
                val_cLoss = 0
                val_cAcc = 0
                val_count = 0
                
                for val_imgs, val_classes in dataloader_val:
                    val_imgs, val_classes = val_imgs.to(device), val_classes.to(device)

                    val_output = overfeat(val_imgs)
                    val_cLoss += F.cross_entropy(val_output, val_classes)

                    _, val_pred = torch.max(val_output, 1)
                    val_cAcc += torch.sum(val_pred == val_classes)

                    val_count += 1

                val_loss = val_cLoss / val_count
                val_accuracy = val_cAcc / val_count

                print(f'\tValidation Loss: {val_loss:.4f} \t Validation Acc: {val_cAcc} / {val_count} ({val_accuracy.item()*100:0f}%)')
                tbwriter.add_scalar('val_loss', val_loss.item(), step)
                tbwriter.add_scalar('val_accuracy', val_accuracy.item(), step)
                overfeat.train()

        step += 1
        
    lr_scheduler.step()
    
    if(Fast):
        checkpoint_path = os.path.join(path_checkpoint, f'overfeat_fast_states_epoch{epoch}.pkl')
    else:
        checkpoint_path = os.path.join(path_checkpoint, f'overfeat_accurate_states_epoch{epoch}.pkl')
    state = {
        'epoch': epoch,
        'step': step,
        'optimizer': optimizer.state_dict(),
        'model': overfeat.state_dict(),
        'seed' : seed
    }
    torch.save(state, checkpoint_path)

Epoch: 1 	Step: 1000 	Loss: 6.9073 	Acc: 0
	Validation Loss: 6.907937526702881 	 Validation Acc: 0.12564103305339813
Epoch: 1 	Step: 2000 	Loss: 6.9111 	Acc: 0
	Validation Loss: 6.908069610595703 	 Validation Acc: 0.12820513546466827
Epoch: 1 	Step: 3000 	Loss: 6.9077 	Acc: 0
	Validation Loss: 6.907866954803467 	 Validation Acc: 0.13076923787593842
Epoch: 1 	Step: 4000 	Loss: 6.8437 	Acc: 0
	Validation Loss: 6.836755752563477 	 Validation Acc: 0.34358975291252136
Epoch: 1 	Step: 5000 	Loss: 6.7401 	Acc: 0
	Validation Loss: 6.7239556312561035 	 Validation Acc: 0.4615384638309479
Epoch: 1 	Step: 6000 	Loss: 6.6208 	Acc: 0
	Validation Loss: 6.631789684295654 	 Validation Acc: 0.7307692766189575
Epoch: 1 	Step: 7000 	Loss: 6.4837 	Acc: 2
	Validation Loss: 6.435487747192383 	 Validation Acc: 1.3615385293960571
Epoch: 1 	Step: 8000 	Loss: 6.1081 	Acc: 4
	Validation Loss: 6.164587497711182 	 Validation Acc: 2.797435998916626
Epoch: 1 	Step: 9000 	Loss: 5.9132 	Acc: 2
	Validation Loss: 5.84160